## 전체데이터 처리

In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

df=pd.read_csv(r'/content/drive/MyDrive/open/데이콘전체전처리.csv')

## 다중공선성이 inf인 것 중에서 상관계수가 1인 것들만 제거

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [28]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

df=pd.read_csv(r'/content/drive/MyDrive/open/0_1_4.csv')

In [ ]:
df.shape

(2400000, 432)

In [29]:
vif_result=pd.read_csv(r'/content/drive/MyDrive/open/vif_result.csv')
vif_result

/usr/local/lib/python3.11/dist-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,feature,VIF
0,_2순위신용체크구분_체크,inf
1,_2순위신용체크구분_없음,inf
2,_2순위신용체크구분_신용,inf
3,직장시도명_충북,inf
4,직장시도명_충남,inf
...,...,...
426,IB문의건수_사용승인내역_R6M,1.373199
427,쇼핑_아울렛_이용금액,1.373157
428,여유_기타이용금액,1.355084
429,당사멤버쉽_방문월수_R6M,1.213710


In [30]:
# 1. VIF가 inf인 feature 추출
inf_features = vif_result[vif_result['VIF'] == np.inf]['feature'].tolist()

In [31]:
# 2. df에서 해당 feature들만 추출 # 85개 처처
df_inf = df[inf_features]
df_inf

,_2순위신용체크구분_체크,_2순위신용체크구분_없음,_2순위신용체크구분_신용,직장시도명_충북,직장시도명_충남,유효카드수_신용,유효카드수_신용체크,유효카드수_체크,보유여부_해외겸용_본인_0,수신거부여부_메일_1,...,할부건수_3M_R12M,직장시도명_경기,할부건수_6M_R12M,할부건수_무이자_3M_R12M,할부건수_유이자_6M_R12M,직장시도명_경남,직장시도명_대구,직장시도명_대전,할부건수_무이자_6M_R12M,할부건수_유이자_3M_R12M
0,True,False,False,False,False,1,2,1,True,False,...,0,False,3,0,0,False,False,False,3,0
1,False,True,False,False,False,1,1,0,True,False,...,0,True,0,0,0,False,False,False,0,0
2,True,False,False,False,False,1,2,1,False,False,...,0,False,3,0,0,False,False,False,3,0
3,True,False,False,False,False,2,3,1,False,False,...,11,False,0,6,0,False,False,False,0,5
4,True,False,False,False,False,1,2,1,False,False,...,0,False,0,0,0,False,False,False,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399995,True,False,False,False,False,1,2,1,False,False,...,0,False,0,0,0,False,False,False,0,0
2399996,False,True,False,False,False,1,1,0,False,False,...,0,False,0,0,0,False,False,False,0,0
2399997,False,True,False,False,False,1,1,0,False,True,...,5,False,0,5,0,False,False,False,0,0
2399998,False,True,False,False,False,1,1,0,True,True,...,0,False,0,0,0,False,False,False,0,0


In [32]:
# 3. 상관계수 행렬 계산
corr_matrix = df_inf.corr().abs()

# 4. 상삼각행렬만 추출 (중복 제거용)
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# 5. 상관계수 1.0인 쌍 추출
high_corr_pairs = upper_tri.stack().reset_index()
high_corr_pairs.columns = ['feature_1', 'feature_2', 'correlation']
high_corr_pairs = high_corr_pairs[np.isclose(high_corr_pairs['correlation'], 1.0)]

In [33]:
high_corr_pairs=high_corr_pairs.sort_values(by='feature_1')
high_corr_pairs

,feature_1,feature_2,correlation
649,보유여부_해외겸용_본인_0,보유여부_해외겸용_본인_1,1.0
2750,보유여부_해외겸용_신용_본인_0,보유여부_해외겸용_신용_본인_1,1.0
1228,수신거부여부_DM_1,수신거부여부_DM_0,1.0
1292,수신거부여부_TM_0,수신거부여부_TM_1,1.0
720,수신거부여부_메일_1,수신거부여부_메일_0,1.0
869,이용가능여부_해외겸용_본인_0,이용가능여부_해외겸용_본인_1,1.0
3105,이용가능여부_해외겸용_신용_본인_1,이용가능여부_해외겸용_신용_본인_0,1.0
1015,이용여부_3M_해외겸용_본인_0,이용여부_3M_해외겸용_본인_1,1.0
3076,이용여부_3M_해외겸용_신용_본인_0,이용여부_3M_해외겸용_신용_본인_1,1.0
3055,청구서발송여부_B0_0,청구서발송여부_B0_1,1.0


In [34]:
# 정렬은 피쳐1로 했기 때문에 피쳐 2 피쳐를 삭제
# 제거할 컬럼 리스트 추출 (feature_2 기준)
drop_cols = high_corr_pairs['feature_2'].unique()

df_model=df.drop(drop_cols,axis=1)

In [35]:
# 제거할 컬럼 리스트
cols_to_remove = [
    '최종유효년월_신용_이용', '직장시도명_경기', '직장시도명_전북', '_2순위신용체크구분_체크',
    '직장시도명_부산', '직장시도명_경북', '직장시도명_제주', '직장시도명_경남',
    '직장시도명_대구', '직장시도명_서울', '직장시도명_전남', '직장시도명_충북',
    '직장시도명_세종', '직장시도명_인천', '직장시도명_울산', '직장시도명_강원',
    '직장시도명_광주', '직장시도명_대전', '직장시도명_충남'
]

# df_model에 존재하는 컬럼만 제거
df_model = df_model.drop(columns=[col for col in cols_to_remove if col in df_model.columns])


# 전체 모델링

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df['Segment_encoded']=le.fit_transform(df['Segment'])

In [ ]:
X= df.drop(['기준년월','ID','Segment','Segment_encoded'],axis=1)
y= df['Segment_encoded']

In [ ]:
# 먼저 설치 필요 (Colab이나 Jupyter 환경에서는 아래 실행 필요)
!pip install xgboost catboost lightgbm --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 68.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.7/291.7 MB 3.7 MB/s eta 0:00:00


In [ ]:
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score, classification_report, confusion_matrix

# 공통 평가 함수
def evaluate_model(name, model, X, y):
    model.fit(X, y)
    y_pred = model.predict(X)
    print(f"\n📌 [{name}] 성능 평가")
    print("✅ F1 Score (macro):", f1_score(y, y_pred, average='macro'))
    print("✅ Confusion Matrix:\n", confusion_matrix(y, y_pred))
    print("✅ Classification Report:\n", classification_report(y, y_pred, target_names=[str(cls) for cls in le.classes_]))
    return model

# 1. LGBM
#lgbm = LGBMClassifier(random_state=42)
#model_lgbm = evaluate_model('LGBM', lgbm, X, y)

# 2. XGBoost
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
model_xgb = evaluate_model('XGBoost', xgb, X, y)

# 3. CatBoost
#cat = CatBoostClassifier(verbose=0, random_state=42)
#model_cat = evaluate_model('CatBoost', cat, X, y)


/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [12:36:57] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



📌 [XGBoost] 성능 평가
✅ F1 Score (macro): 0.8979346439184139
✅ Confusion Matrix:
 [[    972       0       0       0       0]
 [      0     144       0       0       0]
 [      0       0   91206   28464    7856]
 [      0       0   16505  259035   73317]
 [      0       0    4964   41938 1846126]]
✅ Classification Report:
               precision    recall  f1-score   support

           A       1.00      1.00      1.00       972
           B       1.00      1.00      1.00       144
           C       0.81      0.72      0.76    127526
           D       0.79      0.74      0.76    348857
           E       0.96      0.98      0.97   1893028

    accuracy                           0.93   2370527
   macro avg       0.91      0.89      0.90   2370527
weighted avg       0.92      0.93      0.93   2370527



In [ ]:
📌 [XGBoost] 성능 평가
✅ F1 Score (macro): 0.9177117630179101
✅ Confusion Matrix:
 [[1922052       0       0       0       0]
 [      0 1922052       0       0       0]
 [    393       0 1690273  205260   26126]
 [     95       0  223719 1556548  141690]
 [     31       0   22342  171908 1727771]]
✅ Classification Report:
               precision    recall  f1-score   support

           A       1.00      1.00      1.00   1922052
           B       1.00      1.00      1.00   1922052
           C       0.87      0.88      0.88   1922052
           D       0.80      0.81      0.81   1922052
           E       0.91      0.90      0.91   1922052

    accuracy                           0.92   9610260
   macro avg       0.92      0.92      0.92   9610260
weighted avg       0.92      0.92      0.92   9610260

In [ ]:
import pickle

#pickle.dump(model_lgbm, open('/content/drive/MyDrive/open/lgbm_model.pkl', 'wb'))
pickle.dump(model_xgb, open('/content/drive/MyDrive/open/xgb_model_all.pkl', 'wb'))
#pickle.dump(model_cat, open('/content/drive/MyDrive/open/cat_model.pkl', 'wb'))


# C sampling

In [ ]:
# Segment가  C인 데이터 수 확인
target_class='C'
target_count=df_model[df_model['Segment']==target_class].shape[0]

In [ ]:
df_model['Segment'].unique()

array(['D', 'E', 'C', 'A', 'B'], dtype=object)

In [ ]:
from sklearn.utils import resample

#클래스별 분할
dfs = []
for segment in df_model['Segment'].unique():
    class_df = df_model[df_model['Segment']==segment]

    if segment==target_class:
        #A는 그대로 유지
        dfs.append(class_df)
    elif class_df.shape[0] > target_count:
        #수가 많으면 언더샘플링 #replace=False는 중복 방지
        sampled = resample(class_df, replace=False, n_samples=target_count, random_state=10)
        dfs.append(sampled)
    else:
        #수가 적으면 오버샘플링 #replace=True는 중복 허용
        sampled = resample(class_df, replace=True, n_samples=target_count, random_state=10)
        dfs.append(sampled)

In [ ]:
# 클래스 수 맞춘 데이터프레임 합치기
balanced_df= pd.concat(dfs).sample(frac=1,random_state=10).reset_index(drop=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
balanced_df['Segment_encoded']=le.fit_transform(balanced_df['Segment'])

In [ ]:
X= balanced_df.drop(['기준년월','ID','Segment','Segment_encoded'],axis=1)
y= balanced_df['Segment_encoded']

# D sampling

In [36]:
# Segment가 D인 데이터 수 확인
target_class = 'D'
target_count=df_model[df_model['Segment']==target_class].shape[0]

In [37]:
from sklearn.utils import resample

#클래스별 분할
dfs = []
for segment in df_model['Segment'].unique():
    class_df = df_model[df_model['Segment']==segment]

    if segment==target_class:
        #D는 그대로 유지
        dfs.append(class_df)
    elif class_df.shape[0] > target_count:
        #수가 많으면 언더샘플링 #replace=False는 중복 방지
        sampled = resample(class_df, replace=False, n_samples=target_count, random_state=10)
        dfs.append(sampled)
    else:
        #수가 적으면 오버샘플링 #replace=True는 중복 허용
        sampled = resample(class_df, replace=True, n_samples=target_count, random_state=10)
        dfs.append(sampled)

In [38]:
# 클래스 수 맞춘 데이터프레임 합치기
balanced_df = pd.concat(dfs).sample(frac=1, random_state=42).reset_index(drop=True)

In [39]:
from sklearn.preprocessing import LabelEncoder
# 라벨 인코딩
le = LabelEncoder()
balanced_df['Segment_encoded'] = le.fit_transform(balanced_df['Segment'])

In [40]:
# 학습용 데이터 준비
X = balanced_df.drop(['Segment', 'Segment_encoded', 'ID', '기준년월'], axis=1, errors='ignore')
y = balanced_df['Segment_encoded']

In [41]:
print(len(X))

1746210


# E sampling

In [10]:
# Segment가 D인 데이터 수 확인
target_class = 'E'
target_count=df_model[df_model['Segment']==target_class].shape[0]

In [11]:
from sklearn.utils import resample

#클래스별 분할
dfs = []
for segment in df_model['Segment'].unique():
    class_df = df_model[df_model['Segment']==segment]

    if segment==target_class:
        #D는 그대로 유지
        dfs.append(class_df)
    elif class_df.shape[0] > target_count:
        #수가 많으면 언더샘플링 #replace=False는 중복 방지
        sampled = resample(class_df, replace=False, n_samples=target_count, random_state=10)
        dfs.append(sampled)
    else:
        #수가 적으면 오버샘플링 #replace=True는 중복 허용
        sampled = resample(class_df, replace=True, n_samples=target_count, random_state=10)
        dfs.append(sampled)

In [12]:
# 클래스 수 맞춘 데이터프레임 합치기
balanced_df = pd.concat(dfs).sample(frac=1, random_state=42).reset_index(drop=True)

In [14]:
from sklearn.preprocessing import LabelEncoder
# 라벨 인코딩
le = LabelEncoder()
balanced_df['Segment_encoded'] = le.fit_transform(balanced_df['Segment'])

In [15]:
# 학습용 데이터 준비
X = balanced_df.drop(['Segment', 'Segment_encoded', 'ID', '기준년월'], axis=1, errors='ignore')
y = balanced_df['Segment_encoded']

# E 로지스틱

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from scipy.sparse import csr_matrix

In [ ]:
# 로지스틱 분류기 학습
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X, y)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1000, random_state=42)

In [ ]:
# 성능 확인
y_pred = model.predict(X)
print("Classification Report:\n")
print(classification_report(y, y_pred, target_names=le.classes_))

Classification Report:

              precision    recall  f1-score   support

           A       0.89      0.95      0.92   1922052
           B       0.95      1.00      0.97   1922052
           C       0.73      0.63      0.68   1922052
           D       0.67      0.67      0.67   1922052
           E       0.82      0.83      0.83   1922052

    accuracy                           0.82   9610260
   macro avg       0.81      0.82      0.81   9610260
weighted avg       0.81      0.82      0.81   9610260



In [ ]:
import pickle
pickle.dump(model,open('/content/drive/MyDrive/open/logistic_model(inf_상관관계1제거_E)','wb'))

# E light gbm

In [16]:
# 먼저 설치 필요 (Colab이나 Jupyter 환경에서는 아래 실행 필요)
!pip install xgboost catboost lightgbm --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 80.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.7/291.7 MB 3.6 MB/s eta 0:00:00


In [17]:
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score, classification_report, confusion_matrix

# 공통 평가 함수
def evaluate_model(name, model, X, y):
    model.fit(X, y)
    y_pred = model.predict(X)
    print(f"\n📌 [{name}] 성능 평가")
    print("✅ F1 Score (macro):", f1_score(y, y_pred, average='macro'))
    print("✅ Confusion Matrix:\n", confusion_matrix(y, y_pred))
    print("✅ Classification Report:\n", classification_report(y, y_pred, target_names=[str(cls) for cls in le.classes_]))
    return model

# 1. LGBM
lgbm = LGBMClassifier(random_state=42)
model_lgbm = evaluate_model('LGBM', lgbm, X, y)

# 2. XGBoost
#xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
#model_xgb = evaluate_model('XGBoost', xgb, X, y)

# 3. CatBoost
#cat = CatBoostClassifier(verbose=0, random_state=42)
#model_cat = evaluate_model('CatBoost', cat, X, y)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 2.108846 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 57951
[LightGBM] [Info] Number of data points in the train set: 9610260, number of used features: 398
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438

📌 [LGBM] 성능 평가
✅ F1 Score (macro): 0.8979432726246305
✅ Confusion Matrix:
 [[1922052       0       0       0       0]
 [      0 1922052       0       0       0]
 [   6024       0 1604173  275668   36187]
 [    588       0  282990 1476942  161532]
 [     71       1   25646  192973 1703361]]
✅ Classification Report:
               precision    recall  f1-score   support



In [18]:
import pickle

pickle.dump(model_lgbm, open('/content/drive/MyDrive/open/D_lgbm_model.pkl', 'wb'))
#pickle.dump(model_xgb, open('/content/drive/MyDrive/open/xgb_model.pkl', 'wb'))
#pickle.dump(model_cat, open('/content/drive/MyDrive/open/cat_model.pkl', 'wb'))


# E soft voting

In [ ]:
from sklearn.ensemble import VotingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# 개별 모델 정의
lgbm = LGBMClassifier(random_state=42)
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
cat = CatBoostClassifier(verbose=0, random_state=42)

# 앙상블 모델 정의 (soft voting = 확률 기반 평균)
voting_elf = VotingClassifier(
    estimators=[
        ('lgbm', lgbm),
        ('xgb', xgb),
        ('cat', cat)
    ],
    voting='soft'
)

# 모델 학습
voting_elf.fit(X, y)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 2.379942 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 58056
[LightGBM] [Info] Number of data points in the train set: 9610260, number of used features: 417
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438


/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [01:19:45] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


VotingClassifier(estimators=[('lgbm', LGBMClassifier(random_state=42)),
                             ('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric='logloss',
                                            feature_types=None,
                                            feature_weights=None, gamma=None,
                                            grow_policy...
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=None, n_jobs=None,
                                            num_parallel_tree=None, ...)),
                             ('cat',
                              <catboost.core.CatBoostClassifier object at 0x7e6039c141d0>)],
                 voting='soft')

In [ ]:
# 예측
y_pred = voting_elf.predict(X)

# 평가
from sklearn.metrics import f1_score, classification_report, confusion_matrix


print("✅ F1 Score (macro):", f1_score(y, y_pred, average='macro'))
print("✅ Confusion Matrix:\n", confusion_matrix(y, y_pred))
print("✅ Classification Report:\n", classification_report(y, y_pred,target_names = [str(cls) for cls in le.classes_]))

✅ F1 Score (macro): 0.9141857209803002
✅ Confusion Matrix:
 [[1922052       0       0       0       0]
 [      0 1922052       0       0       0]
 [   2405       0 1673804  217410   28433]
 [    342       0  235301 1542520  143889]
 [     37       2   21201  176425 1724387]]
✅ Classification Report:
               precision    recall  f1-score   support

           A       1.00      1.00      1.00   1922052
           B       1.00      1.00      1.00   1922052
           C       0.87      0.87      0.87   1922052
           D       0.80      0.80      0.80   1922052
           E       0.91      0.90      0.90   1922052

    accuracy                           0.91   9610260
   macro avg       0.91      0.91      0.91   9610260
weighted avg       0.91      0.91      0.91   9610260



In [ ]:
import pickle
# 모델 저장
pickle.dump(voting_elf,open('/content/drive/MyDrive/open/voting_model(inf_상관관계1제거_E)','wb'))

# D SVM

In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from tqdm import tqdm

# tqdm으로 진행률 보면서 학습 (여기선 1번 루프지만 확장 가능)
for _ in tqdm(range(1), desc="🚀 Training SVM"):
    svm_pipeline = make_pipeline(
        StandardScaler(),
        SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)
    )
    svm_pipeline.fit(X, y)

# 예측 및 평가
y_pred = svm_pipeline.predict(X)

print(f"\n📌 [SVM] 성능 평가")
print("✅ F1 Score (macro):", f1_score(y, y_pred, average='macro'))
print("✅ Confusion Matrix:\n", confusion_matrix(y, y_pred))
print("✅ Classification Report:\n", classification_report(y, y_pred, target_names=[str(cls) for cls in le.classes_]))


🚀 Training SVM:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
import pickle

pickle.dump(svm_pipeline, open('/content/drive/MyDrive/open/svm_pipeline_model.pkl', 'wb'))

In [ ]:
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
f1_scores = []

for train_idx, val_idx in tqdm(skf.split(X, y), desc="🚀 SVM CV Training"):
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    svm = make_pipeline(StandardScaler(), SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42))
    svm.fit(X_train, y_train)
    y_pred = svm.predict(X_val)

    score = f1_score(y_val, y_pred, average='macro')
    f1_scores.append(score)

print("✅ 평균 F1 Score (macro):", sum(f1_scores) / len(f1_scores))


# D logistic

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from scipy.sparse import csr_matrix

In [ ]:
# 로지스틱 분류기 학습
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X, y)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1000, random_state=42)

In [ ]:
# 성능 확인
y_pred = model.predict(X)
print("Classification Report:\n")
print(classification_report(y, y_pred, target_names=le.classes_))

Classification Report:

              precision    recall  f1-score   support

           A       0.89      0.94      0.92    349242
           B       0.95      1.00      0.97    349242
           C       0.72      0.63      0.67    349242
           D       0.67      0.67      0.67    349242
           E       0.82      0.84      0.83    349242

    accuracy                           0.82   1746210
   macro avg       0.81      0.82      0.81   1746210
weighted avg       0.81      0.82      0.81   1746210



In [ ]:
import pickle
pickle.dump(model,open('/content/drive/MyDrive/open/logistic_model(inf_상관관계1제거_D)','wb'))

# D 모델링

In [ ]:
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score, classification_report, confusion_matrix

# 공통 평가 함수
def evaluate_model(name, model, X, y):
    model.fit(X, y)
    y_pred = model.predict(X)
    print(f"\n📌 [{name}] 성능 평가")
    print("✅ F1 Score (macro):", f1_score(y, y_pred, average='macro'))
    print("✅ Confusion Matrix:\n", confusion_matrix(y, y_pred))
    print("✅ Classification Report:\n", classification_report(y, y_pred, target_names=[str(cls) for cls in le.classes_]))
    return model

# 1. LGBM
lgbm = LGBMClassifier(random_state=42)
model_lgbm = evaluate_model('LGBM', lgbm, X, y)

# 2. XGBoost
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
model_xgb = evaluate_model('XGBoost', xgb, X, y)

# 3. CatBoost
cat = CatBoostClassifier(verbose=0, random_state=42)
model_cat = evaluate_model('CatBoost', cat, X, y)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 2.057142 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 58056
[LightGBM] [Info] Number of data points in the train set: 9610260, number of used features: 417
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438

📌 [LGBM] 성능 평가
✅ F1 Score (macro): 0.8981102158617743
✅ Confusion Matrix:
 [[1922052       0       0       0       0]
 [      0 1922052       0       0       0]
 [   5546       0 1605908  273617   36981]
 [    540       0  282828 1476868  161816]
 [     63       2   25422  193201 1703364]]
✅ Classification Report:
               precision    recall  f1-score   support



/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [02:37:00] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



📌 [XGBoost] 성능 평가
✅ F1 Score (macro): 0.9177117630179101
✅ Confusion Matrix:
 [[1922052       0       0       0       0]
 [      0 1922052       0       0       0]
 [    393       0 1690273  205260   26126]
 [     95       0  223719 1556548  141690]
 [     31       0   22342  171908 1727771]]
✅ Classification Report:
               precision    recall  f1-score   support

           A       1.00      1.00      1.00   1922052
           B       1.00      1.00      1.00   1922052
           C       0.87      0.88      0.88   1922052
           D       0.80      0.81      0.81   1922052
           E       0.91      0.90      0.91   1922052

    accuracy                           0.92   9610260
   macro avg       0.92      0.92      0.92   9610260
weighted avg       0.92      0.92      0.92   9610260


📌 [CatBoost] 성능 평가
✅ F1 Score (macro): 0.9174517255887208
✅ Confusion Matrix:
 [[1922052       0       0       0       0]
 [      0 1922052       0       0       0]
 [   8582      19 168633

In [ ]:
import pickle

pickle.dump(model_lgbm, open('/content/drive/MyDrive/open/lgbm_model.pkl', 'wb'))
pickle.dump(model_xgb, open('/content/drive/MyDrive/open/xgb_model.pkl', 'wb'))
pickle.dump(model_cat, open('/content/drive/MyDrive/open/cat_model.pkl', 'wb'))


# D soft voting

In [ ]:
from sklearn.ensemble import VotingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# 개별 모델 정의
lgbm = LGBMClassifier(random_state=42)
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
cat = CatBoostClassifier(verbose=0, random_state=42)

# 앙상블 모델 정의 (soft voting = 확률 기반 평균)
voting_dlf = VotingClassifier(
    estimators=[
        ('lgbm', lgbm),
        ('xgb', xgb),
        ('cat', cat)
    ],
    voting='soft'
)

# 모델 학습
voting_dlf.fit(X, y)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 3.561291 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 57944
[LightGBM] [Info] Number of data points in the train set: 1746210, number of used features: 398
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438


/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [06:41:58] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


VotingClassifier(estimators=[('lgbm', LGBMClassifier(random_state=42)),
                             ('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric='logloss',
                                            feature_types=None,
                                            feature_weights=None, gamma=None,
                                            grow_policy...
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=None, n_jobs=None,
                                            num_parallel_tree=None, ...)),
                             ('cat',
                              <catboost.core.CatBoostClassifier object at 0x798f34528ed0>)],
                 voting='soft')

In [ ]:
# 예측
y_pred = voting_dlf.predict(X)

# 평가
from sklearn.metrics import f1_score, classification_report, confusion_matrix


print("✅ F1 Score (macro):", f1_score(y, y_pred, average='macro'))
print("✅ Confusion Matrix:\n", confusion_matrix(y, y_pred))
print("✅ Classification Report:\n", classification_report(y, y_pred,target_names = [str(cls) for cls in le.classes_]))

✅ F1 Score (macro): 0.9131415175306812
✅ Confusion Matrix:
 [[349242      0      0      0      0]
 [     0 349242      0      0      0]
 [   570      0 303226  40235   5211]
 [    82      0  43535 279240  26385]
 [     5      0   3737  32024 313476]]
✅ Classification Report:
               precision    recall  f1-score   support

           A       1.00      1.00      1.00    349242
           B       1.00      1.00      1.00    349242
           C       0.87      0.87      0.87    349242
           D       0.79      0.80      0.80    349242
           E       0.91      0.90      0.90    349242

    accuracy                           0.91   1746210
   macro avg       0.91      0.91      0.91   1746210
weighted avg       0.91      0.91      0.91   1746210



In [ ]:
✅ F1 Score (macro): 0.9131415175306812
✅ Confusion Matrix:
 [[349242      0      0      0      0]
 [     0 349242      0      0      0]
 [   570      0 303226  40235   5211]
 [    82      0  43535 279240  26385]
 [     5      0   3737  32024 313476]]
✅ Classification Report:
               precision    recall  f1-score   support

           A       1.00      1.00      1.00    349242
           B       1.00      1.00      1.00    349242
           C       0.87      0.87      0.87    349242
           D       0.79      0.80      0.80    349242
           E       0.91      0.90      0.90    349242

    accuracy                           0.91   1746210
   macro avg       0.91      0.91      0.91   1746210
weighted avg       0.91      0.91      0.91   1746210

In [ ]:
import pickle
# 모델 저장
pickle.dump(voting_dlf,open('/content/drive/MyDrive/open/voting_model(inf_상관관계1제거_D)','wb'))

# D soft voting tuning

In [24]:
from sklearn.ensemble import VotingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# 1. 개별 모델 정의 (튜닝 포함)
lgbm = LGBMClassifier(n_estimators=200, learning_rate=0.05, max_depth=7, random_state=42)
xgb = XGBClassifier(n_estimators=200, learning_rate=0.05, max_depth=6,
                    use_label_encoder=False, eval_metric='logloss', random_state=42)
cat = CatBoostClassifier(iterations=200, learning_rate=0.05, depth=6, verbose=0, random_state=42)

# 2. Soft Voting 앙상블 (가중치 부여)
voting_clf = VotingClassifier(
    estimators=[('lgbm', lgbm), ('xgb', xgb), ('cat', cat)],
    voting='soft',
    weights=[1.5, 1, 1.2]  # lgbm > catboost > xgb
)

# 3. 학습
voting_clf.fit(X, y)

# 4. 예측 및 평가
y_pred = voting_clf.predict(X)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 1.007318 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 57944
[LightGBM] [Info] Number of data points in the train set: 1746210, number of used features: 398
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438


/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [01:51:25] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [25]:
from sklearn.metrics import f1_score, classification_report, confusion_matrix
print("✅ F1 Score(macro):", f1_score(y, y_pred, average='macro'))
print("✅ Confusion Matrix:\n", confusion_matrix(y, y_pred))
print("✅ Classification Report:\n", classification_report(y, y_pred))


✅ F1 Score(macro): 0.8867156611803685
✅ Confusion Matrix:
 [[349242      0      0      0      0]
 [     0 349242      0      0      0]
 [  6024     31 280744  54121   8322]
 [   328     12  53922 261709  33271]
 [    27      1   4481  36826 307907]]
✅ Classification Report:
               precision    recall  f1-score   support

           0       0.98      1.00      0.99    349242
           1       1.00      1.00      1.00    349242
           2       0.83      0.80      0.82    349242
           3       0.74      0.75      0.75    349242
           4       0.88      0.88      0.88    349242

    accuracy                           0.89   1746210
   macro avg       0.89      0.89      0.89   1746210
weighted avg       0.89      0.89      0.89   1746210



In [26]:
import pickle
# 모델 저장
pickle.dump(voting_clf,open('/content/drive/MyDrive/open/voting_model(inf_상관관계1제거_D)_tuning','wb'))

#C 앙상블 모델(Soft Voting)

In [ ]:
# 먼저 설치 필요 (Colab이나 Jupyter 환경에서는 아래 실행 필요)
!pip install xgboost catboost lightgbm --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 76.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.7/291.7 MB 3.6 MB/s eta 0:00:00


In [ ]:
from sklearn.ensemble import VotingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# 개별 모델 정의
lgbm = LGBMClassifier(random_state=42)
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
cat = CatBoostClassifier(verbose=0, random_state=42)

# 앙상블 모델 정의 (soft voting = 확률 기반 평균)
voting_clf = VotingClassifier(
    estimators=[
        ('lgbm', lgbm),
        ('xgb', xgb),
        ('cat', cat)
    ],
    voting='soft'
)

# 모델 학습
voting_clf.fit(X, y)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.478480 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 58101
[LightGBM] [Info] Number of data points in the train set: 637950, number of used features: 417
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438


/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [11:25:46] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


VotingClassifier(estimators=[('lgbm', LGBMClassifier(random_state=42)),
                             ('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric='logloss',
                                            feature_types=None,
                                            feature_weights=None, gamma=None,
                                            grow_policy...
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=None, n_jobs=None,
                                            num_parallel_tree=None, ...)),
                             ('cat',
                              <catboost.core.CatBoostClassifier object at 0x785806407610>)],
                 voting='soft')

In [ ]:
# 예측
y_pred = voting_clf.predict(X)

# 평가
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

print("✅ Confusion Matrix:\n", confusion_matrix(y, y_pred))
print("✅ Classification Report:\n", classification_report(y, y_pred,target_names=le.classes_))

✅ Confusion Matrix:
 [[127590      0      0      0      0]
 [     0 127590      0      0      0]
 [   271      0 110057  15235   2027]
 [    28      0  15701 102537   9324]
 [     2      0   1402  11136 115050]]
✅ Classification Report:
               precision    recall  f1-score   support

           A       1.00      1.00      1.00    127590
           B       1.00      1.00      1.00    127590
           C       0.87      0.86      0.86    127590
           D       0.80      0.80      0.80    127590
           E       0.91      0.90      0.91    127590

    accuracy                           0.91    637950
   macro avg       0.91      0.91      0.91    637950
weighted avg       0.91      0.91      0.91    637950



In [ ]:
import pickle
# 모델 저장
pickle.dump(voting_clf,open('/content/drive/MyDrive/open/voting_model(inf_상관관계1제거_C)','wb'))

# xgb 제거

In [ ]:
from sklearn.ensemble import VotingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# 개별 모델 정의
lgbm = LGBMClassifier(random_state=42)
#xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
cat = CatBoostClassifier(verbose=0, random_state=42)

# 앙상블 모델 정의 (soft voting = 확률 기반 평균)
voting_clf = VotingClassifier(
    estimators=[
        ('lgbm', lgbm),
        #('xgb', xgb),
        ('cat', cat)
    ],
    voting='soft'
)

# 모델 학습
voting_clf.fit(X, y)
# 예측
y_pred = voting_clf.predict(X)

# 평가
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

print("✅ Confusion Matrix:\n", confusion_matrix(y, y_pred))
print("✅ Classification Report:\n", classification_report(y, y_pred,target_names=le.classes_))

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.488242 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 58101
[LightGBM] [Info] Number of data points in the train set: 637950, number of used features: 417
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
✅ Confusion Matrix:
 [[127590      0      0      0      0]
 [     0 127590      0      0      0]
 [   667      6 108272  16468   2177]
 [    45      0  16875 100763   9907]
 [     3      0   1454  11777 114356]]
✅ Classification Report:
               precision    recall  f1-score   support

           A       0.99      1.00      1.00    127590
           B       1.00      1.00      1.00    127590
           C       0.86      0.85    

# 소프트보팅 튜닝

In [ ]:
from sklearn.ensemble import VotingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# 1. 개별 모델 정의 (튜닝 포함)
lgbm = LGBMClassifier(n_estimators=200, learning_rate=0.05, max_depth=7, random_state=42)
xgb = XGBClassifier(n_estimators=200, learning_rate=0.05, max_depth=6,
                    use_label_encoder=False, eval_metric='logloss', random_state=42)
cat = CatBoostClassifier(iterations=200, learning_rate=0.05, depth=6, verbose=0, random_state=42)

# 2. Soft Voting 앙상블 (가중치 부여)
voting_clf = VotingClassifier(
    estimators=[('lgbm', lgbm), ('xgb', xgb), ('cat', cat)],
    voting='soft',
    weights=[1.5, 1, 1.2]  # lgbm > catboost > xgb
)

# 3. 학습
voting_clf.fit(X, y)

# 4. 예측 및 평가
y_pred = voting_clf.predict(X)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.467680 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 58101
[LightGBM] [Info] Number of data points in the train set: 637950, number of used features: 417
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438


/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [15:31:39] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

In [ ]:
from sklearn.metrics import f1_score, classification_report, confusion_matrix
print("✅ F1 Score(macro):", f1_score(y, y_pred, average='macro'))
print("✅ Confusion Matrix:\n", confusion_matrix(y, y_pred))
print("✅ Classification Report:\n", classification_report(y, y_pred))


✅ F1 Score(macro): 0.8876474110208061
✅ Confusion Matrix:
 [[127590      0      0      0      0]
 [     0 127590      0      0      0]
 [  2180     21 102412  19889   3088]
 [   131      1  19473  96140  11845]
 [    12      0   1617  13269 112692]]
✅ Classification Report:
               precision    recall  f1-score   support

           0       0.98      1.00      0.99    127590
           1       1.00      1.00      1.00    127590
           2       0.83      0.80      0.82    127590
           3       0.74      0.75      0.75    127590
           4       0.88      0.88      0.88    127590

    accuracy                           0.89    637950
   macro avg       0.89      0.89      0.89    637950
weighted avg       0.89      0.89      0.89    637950



In [ ]:
import pickle
# 모델 저장
pickle.dump(voting_clf,open('/content/drive/MyDrive/open/voting_model(inf_상관관계1제거_C)_tuning','wb'))

# 튜닝2

In [ ]:
from sklearn.ensemble import VotingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# 개별 모델 정의
lgbm = LGBMClassifier(random_state=42)
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
cat = CatBoostClassifier(verbose=0, random_state=42)

# 앙상블 모델 정의 (soft voting = 확률 기반 평균)
voting_clf = VotingClassifier(
    estimators=[
        ('lgbm', lgbm),
        ('xgb', xgb),
        ('cat', cat)
    ],
    voting='soft',
    weights=[1, 1, 1]  # 균형 잡기
)

# 모델 학습
voting_clf.fit(X, y)
# 예측
y_pred = voting_clf.predict(X)

# 평가
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

print("✅ Confusion Matrix:\n", confusion_matrix(y, y_pred))
print("✅ Classification Report:\n", classification_report(y, y_pred,target_names=le.classes_))

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.464766 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 58101
[LightGBM] [Info] Number of data points in the train set: 637950, number of used features: 417
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438


/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [15:53:33] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


✅ Confusion Matrix:
 [[127590      0      0      0      0]
 [     0 127590      0      0      0]
 [   271      0 110057  15235   2027]
 [    28      0  15701 102537   9324]
 [     2      0   1402  11136 115050]]
✅ Classification Report:
               precision    recall  f1-score   support

           A       1.00      1.00      1.00    127590
           B       1.00      1.00      1.00    127590
           C       0.87      0.86      0.86    127590
           D       0.80      0.80      0.80    127590
           E       0.91      0.90      0.91    127590

    accuracy                           0.91    637950
   macro avg       0.91      0.91      0.91    637950
weighted avg       0.91      0.91      0.91    637950



In [ ]:
import pickle
# 모델 저장
pickle.dump(voting_clf,open('/content/drive/MyDrive/open/voting_model(inf_상관관계1제거_C)_tuning2','wb'))

# 튜닝3

In [ ]:
!pip install optuna lightgbm xgboost catboost --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 583.9/583.9 kB 33.3 MB/s eta 0:00:00


In [ ]:
import optuna
from sklearn.metrics import f1_score
from sklearn.ensemble import VotingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold

def objective(trial):
    # 하이퍼파라미터 샘플링
    lgbm_params = {
        'n_estimators': trial.suggest_int('lgbm_n', 100, 300),
        'learning_rate': trial.suggest_float('lgbm_lr', 0.01, 0.2),
        'max_depth': trial.suggest_int('lgbm_depth', 3, 10)
    }
    xgb_params = {
        'n_estimators': trial.suggest_int('xgb_n', 100, 300),
        'learning_rate': trial.suggest_float('xgb_lr', 0.01, 0.2),
        'max_depth': trial.suggest_int('xgb_depth', 3, 10),
        'use_label_encoder': False,
        'eval_metric': 'logloss'
    }
    cat_params = {
        'iterations': trial.suggest_int('cat_n', 100, 300),
        'learning_rate': trial.suggest_float('cat_lr', 0.01, 0.2),
        'depth': trial.suggest_int('cat_depth', 3, 10),
        'verbose': 0
    }

    # 가중치 튜닝
    w1 = trial.suggest_float("w1", 0.5, 2.0)
    w2 = trial.suggest_float("w2", 0.5, 2.0)
    w3 = trial.suggest_float("w3", 0.5, 2.0)

    # 모델 구성
    lgbm = LGBMClassifier(**lgbm_params, random_state=42)
    xgb = XGBClassifier(**xgb_params, random_state=42)
    cat = CatBoostClassifier(**cat_params, random_state=42)

    voting = VotingClassifier(
        estimators=[('lgbm', lgbm), ('xgb', xgb), ('cat', cat)],
        voting='soft',
        weights=[w1, w2, w3]
    )

    # 교차검증 (macro F1)
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    scores = cross_val_score(voting, X, y, cv=cv, scoring='f1_macro', n_jobs=-1)

    return scores.mean()

# Optuna 실행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)  # 원하는 탐색 횟수

# 결과 출력
print("🎯 Best Macro F1:", study.best_value)
print("✅ Best Params:", study.best_params)


[I 2025-04-23 16:08:54,317] A new study created in memory with name: no-name-58c22f90-d671-4b13-9508-d5259e0ffc22
[I 2025-04-23 16:41:53,525] Trial 0 finished with value: 0.8803843514538953 and parameters: {'lgbm_n': 273, 'lgbm_lr': 0.04738368728897198, 'lgbm_depth': 3, 'xgb_n': 269, 'xgb_lr': 0.10607850014167058, 'xgb_depth': 3, 'cat_n': 281, 'cat_lr': 0.08525935615178308, 'cat_depth': 8, 'w1': 0.7450874242927936, 'w2': 1.2452897126061502, 'w3': 1.8527661119171046}. Best is trial 0 with value: 0.8803843514538953.
[I 2025-04-23 17:31:28,629] Trial 1 finished with value: 0.8842376467168606 and parameters: {'lgbm_n': 126, 'lgbm_lr': 0.18671756927460204, 'lgbm_depth': 10, 'xgb_n': 116, 'xgb_lr': 0.04934251751911591, 'xgb_depth': 5, 'cat_n': 112, 'cat_lr': 0.07479998850891784, 'cat_depth': 7, 'w1': 1.293439210391814, 'w2': 1.6639689770991657, 'w3': 0.9485145275926731}. Best is trial 1 with value: 0.8842376467168606.
[I 2025-04-23 18:14:49,408] Trial 2 finished with value: 0.890529034268124

In [ ]:
#[I 2025-04-23 18:43:54,312] Trial 3 finished with value: 0.914100013167454 and parameters: {'lgbm_n': 127, 'lgbm_lr': 0.13485637021400995, 'lgbm_depth': 3, 'xgb_n': 229, 'xgb_lr': 0.15005518396099266, 'xgb_depth': 10, 'cat_n': 235, 'cat_lr': 0.12525589838517714, 'cat_depth': 4, 'w1': 1.1473761567337295, 'w2': 1.6103316713739315, 'w3': 0.7467788305024395}. Best is trial 3 with value: 0.914100013167454.

In [ ]:
from sklearn.ensemble import VotingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# Optuna에서 얻은 최적 파라미터 수동 세팅
best_params = {
    'lgbm_n': 127,
    'lgbm_lr': 0.13485637021400995,
    'lgbm_depth': 3,
    'xgb_n': 229,
    'xgb_lr': 0.15005518396099266,
    'xgb_depth': 10,
    'cat_n': 235,
    'cat_lr': 0.12525589838517714,
    'cat_depth': 4,
    'w1': 1.1473761567337295,
    'w2': 1.6103316713739315,
    'w3': 0.7467788305024395
}

# 각 개별 모델 정의
lgbm = LGBMClassifier(
    n_estimators=best_params['lgbm_n'],
    learning_rate=best_params['lgbm_lr'],
    max_depth=best_params['lgbm_depth'],
    random_state=42
)

xgb = XGBClassifier(
    n_estimators=best_params['xgb_n'],
    learning_rate=best_params['xgb_lr'],
    max_depth=best_params['xgb_depth'],
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42
)

cat = CatBoostClassifier(
    iterations=best_params['cat_n'],
    learning_rate=best_params['cat_lr'],
    depth=best_params['cat_depth'],
    verbose=0,
    random_state=42
)

# Soft Voting 앙상블 구성
voting_final = VotingClassifier(
    estimators=[('lgbm', lgbm), ('xgb', xgb), ('cat', cat)],
    voting='soft',
    weights=[best_params['w1'], best_params['w2'], best_params['w3']]
)

# 모델 학습
voting_final.fit(X, y)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.413523 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 58101
[LightGBM] [Info] Number of data points in the train set: 637950, number of used features: 417
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Info] Start training from score -1.609438
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [23:58:03] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


VotingClassifier(estimators=[('lgbm',
                              LGBMClassifier(learning_rate=0.13485637021400995,
                                             max_depth=3, n_estimators=127,
                                             random_state=42)),
                             ('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric='logloss'...
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=10,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=229, n_jobs=None,
                                            num_parallel_tree=None, ...)),
                             ('cat',
                              <catboost.core.CatBoostClassifier object at 0x7e6044bbc3d0>)],
                 voting='soft',
                 weights=[1.1473761567337295, 1.6103316713739315,
                          0.7467788305024395])

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, f1_score

y_pred = voting_final.predict(X)

print("✅ F1 Score (macro):", f1_score(y, y_pred, average='macro'))
print("✅ Confusion Matrix:\n", confusion_matrix(y, y_pred))
print("✅ Classification Report:\n", classification_report(y, y_pred, target_names=le.classes_))


✅ F1 Score (macro): 0.9720700557977165
✅ Confusion Matrix:
 [[127590      0      0      0      0]
 [     0 127590      0      0      0]
 [   545      2 122408   3513   1122]
 [    82      0   4038 119234   4236]
 [     8      0    538   3709 123335]]
✅ Classification Report:
               precision    recall  f1-score   support

           A       1.00      1.00      1.00    127590
           B       1.00      1.00      1.00    127590
           C       0.96      0.96      0.96    127590
           D       0.94      0.93      0.94    127590
           E       0.96      0.97      0.96    127590

    accuracy                           0.97    637950
   macro avg       0.97      0.97      0.97    637950
weighted avg       0.97      0.97      0.97    637950



In [ ]:
✅ Confusion Matrix:
 [[127590      0      0      0      0]
 [     0 127590      0      0      0]
 [   271      0 110057  15235   2027]
 [    28      0  15701 102537   9324]
 [     2      0   1402  11136 115050]]
✅ Classification Report:
               precision    recall  f1-score   support

           A       1.00      1.00      1.00    127590
           B       1.00      1.00      1.00    127590
           C       0.87      0.86      0.86    127590
           D       0.80      0.80      0.80    127590
           E       0.91      0.90      0.91    127590

    accuracy                           0.91    637950
   macro avg       0.91      0.91      0.91    637950
weighted avg       0.91      0.91      0.91    637950

In [ ]:
import pickle
# 모델 저장
pickle.dump(voting_final,open('/content/drive/MyDrive/open/voting_final_model(inf_상관관계1제거_C)','wb'))

# Stacking Ensemble

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

base_models = [
    ('lgbm', LGBMClassifier(random_state=42)),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)),
    ('cat', CatBoostClassifier(verbose=0, random_state=42))
]

meta_model = LogisticRegression()

stacking_clf = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_model,
    cv=5,
    n_jobs=-1
)

stacking_clf.fit(X, y)


NameError: name 'X_test' is not defined

In [ ]:
y_pred = stacking_clf.predict(X)

# 평가
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

print("✅ Confusion Matrix:\n", confusion_matrix(y, y_pred))
print("✅ Classification Report:\n", classification_report(y, y_pred))

✅ Confusion Matrix:
 [[127590      0      0      0      0]
 [     0 127590      0      0      0]
 [    11      0 113153  13073   1353]
 [     3      0  12339 106771   8477]
 [     1      0   1019  10412 116158]]
✅ Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    127590
           1       1.00      1.00      1.00    127590
           2       0.89      0.89      0.89    127590
           3       0.82      0.84      0.83    127590
           4       0.92      0.91      0.92    127590

    accuracy                           0.93    637950
   macro avg       0.93      0.93      0.93    637950
weighted avg       0.93      0.93      0.93    637950



In [ ]:
import pickle
pickle.dump(stacking_clf,open('/content/drive/MyDrive/open/stacking_model(inf_상관관계1제거_C)','wb'))

# Bagging

In [ ]:
from sklearn.ensemble import BaggingClassifier
from lightgbm import LGBMClassifier

bagging_clf = BaggingClassifier(
    estimator=LGBMClassifier(random_state=42),
    n_estimators=10,
    random_state=42,
    n_jobs=-1
)

bagging_clf.fit(X, y)
y_pred = bagging_clf.predict(X)


In [ ]:
print("✅ Confusion Matrix:\n", confusion_matrix(y, y_pred))
print("✅ Classification Report:\n", classification_report(y, y_pred))

✅ Confusion Matrix:
 [[127590      0      0      0      0]
 [     0 127590      0      0      0]
 [   497      1 106490  18138   2464]
 [    40      0  18429  98817  10304]
 [     4      0   1635  12445 113506]]
✅ Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    127590
           1       1.00      1.00      1.00    127590
           2       0.84      0.83      0.84    127590
           3       0.76      0.77      0.77    127590
           4       0.90      0.89      0.89    127590

    accuracy                           0.90    637950
   macro avg       0.90      0.90      0.90    637950
weighted avg       0.90      0.90      0.90    637950



In [ ]:
import pickle
pickle.dump(bagging_clf,open('/content/drive/MyDrive/open/bagging_model(inf_상관관계1제거_C)','wb'))

# 랜덤 포레스트

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 1. 모델 정의
rf_model = RandomForestClassifier(
    n_estimators=100,        # 트리 개수
    max_depth=None,          # 트리 최대 깊이 (None이면 자동)
    random_state=42,
    n_jobs=-1                # 병렬 처리
)

# 2. 학습
rf_model.fit(X, y)

# 3. 예측
y_pred = rf_model.predict(X)

# 4. 평가
print("✅ Confusion Matrix:\n", confusion_matrix(y, y_pred))
print("✅ Classification Report:\n", classification_report(y, y_pred))


✅ Confusion Matrix:
 [[127590      0      0      0      0]
 [     0 127590      0      0      0]
 [     0      0 127590      0      0]
 [     0      0      0 127589      1]
 [     0      0      0      0 127590]]
✅ Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    127590
           1       1.00      1.00      1.00    127590
           2       1.00      1.00      1.00    127590
           3       1.00      1.00      1.00    127590
           4       1.00      1.00      1.00    127590

    accuracy                           1.00    637950
   macro avg       1.00      1.00      1.00    637950
weighted avg       1.00      1.00      1.00    637950



In [ ]:
pickle.dump(rf_model,open('/content/drive/MyDrive/open/rf_model(inf_상관관계1제거_C)','wb'))